In [2]:
import numpy as np
np.__version__

'2.2.2'

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Embedding,
    LSTM,
    Dense,
    Bidirectional,
    Attention,
    TextVectorization,
)
from tensorflow.keras.models import Model
import numpy as np


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/admin/anaconda3/envs/venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/admin/anaconda3/envs/venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/admin/anaconda3/envs/venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

In [2]:
import string
import re

In [3]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")


vocab_size = 15000
sequence_length = 20

In [4]:
vocab_size = 20000
sequence_length = 20

In [5]:
port = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

span = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

eng = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

I0000 00:00:1741967949.521035    2400 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4309 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 6GB Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [6]:
import numpy as np


def read_vec_file(file):
    # Initialize an empty dictionary to hold the embeddings
    embeddings = {}

    # Open the .vec file
    with open(file, "r", encoding="utf-8") as f:
        # Read the first line to get the number of vectors and their dimensions
        first_line = f.readline().strip()
        vocab_size, vector_dim = map(int, first_line.split())

        # Iterate over each subsequent line
        for line in f:
            # Split the line into the word and its vector components
            parts = line.strip().split()
            word = parts[0]
            vector = np.array(parts[1:], dtype=np.float16)
            embeddings[word] = vector
    return embeddings


# Access a word vector

In [7]:
EMBEDDING_DIM = 300  # Word embedding dimension
LSTM_UNITS = 512  # LSTM hidden units
VOCAB_SIZE = 10000  # Vocabulary size
MAX_LEN = 20  # Max sequence length

In [14]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, lstm_units):
        super(Encoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.lstm = Bidirectional(
            LSTM(lstm_units, return_sequences=True, return_state=True)
        )

    def call(self, inputs):
        x = self.embedding(inputs)
        output, forward_h, forward_c, backward_h, backward_c = self.lstm(inputs)
        # Combine forward and backward states
        state_h = tf.concat([forward_h, backward_h], axis=-1)
        state_c = tf.concat([forward_c, backward_c], axis=-1)
        return output, state_h, state_c  # Return sequence outputs and final state

In [9]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)  # Encoder output transformation
        self.W2 = Dense(units)  # Decoder hidden state transformation
        self.V = Dense(1)  # Attention score

    def call(self, encoder_output, decoder_hidden):
        # Expand decoder hidden state for broadcasting
        decoder_hidden_with_time_axis = tf.expand_dims(decoder_hidden, 1)
        # Calculate attention scores
        score = self.V(
            tf.nn.tanh(self.W1(encoder_output) + self.W2(decoder_hidden_with_time_axis))
        )
        attention_weights = tf.nn.softmax(score, axis=1)  # Softmax across time axis
        context_vector = attention_weights * encoder_output
        context_vector = tf.reduce_sum(context_vector, axis=1)  # Sum across time axis
        return context_vector, attention_weights

In [13]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, lstm_units):
        super(Decoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.lstm = Bidirectional(
            LSTM(lstm_units, return_sequences=True, return_state=True)
        )
        self.fc = Dense(vocab_size, activation="softmax")
        self.attention = BahdanauAttention(lstm_units)

    def call(self, inputs, encoder_output, hidden_state):
        x = self.embedding(inputs)  # Shape: (batch_size, seq_len, embedding_dim)
        context_vectors = []  # Store attention context for each timestep

        for t in range(inputs.shape[1]):  # Iterate over time steps
            context_vector, _ = self.attention(encoder_output, hidden_state)
            context_vectors.append(context_vector)

        context_vectors = tf.stack(
            context_vectors, axis=1
        )  # Shape: (batch_size, seq_len, lstm_units)

        inputs = tf.concat(
            [context_vectors, inputs], axis=-1
        )  # Concatenate across last axis

        output, forward_h, forward_c, backward_h, backward_c = self.lstm(inputs)
        state_h = tf.concat([forward_h, backward_h], axis=-1)
        state_c = tf.concat([forward_c, backward_c], axis=-1)

        output = self.fc(output)  # Convert to vocabulary predictions
        return output, state_h, state_c

In [12]:
class Seq2Seq(Model):
    def __init__(self, vocab_size, embedding_dim, lstm_units):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(vocab_size, embedding_dim, lstm_units)
        self.decoder = Decoder(vocab_size, embedding_dim, lstm_units)

    def call(self, inputs):
        encoder_input, decoder_input = inputs
        encoder_output, encoder_h, encoder_c = self.encoder(encoder_input)
        decoder_output, _, _ = self.decoder(decoder_input, encoder_output, encoder_h)
        return decoder_output

    def inference(self, inputs):
        encoder_output, encoder_h, encoder_c = self.encoder(inputs)
        decoder_output, _, _ = self.decoder(decoder_input, encoder_output, encoder_h)
        return decoder_output

In [ ]:
# Instantiate the model
model = Seq2Seq(VOCAB_SIZE, EMBEDDING_DIM, LSTM_UNITS)

# Compile
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Generate dummy data (replace with actual dataset)
encoder_input = np.random.randint(0, VOCAB_SIZE, (64, MAX_LEN)).astype(
    "float32"
)  # (batch_size, seq_len)
decoder_input = np.random.randint(0, VOCAB_SIZE, (64, MAX_LEN)).astype(
    "float32"
)  # (batch_size, seq_len)
decoder_target = np.random.randint(0, VOCAB_SIZE, (64, MAX_LEN)).astype(
    "float32"
)  # Shifted targets

# Train the model
model.fit([encoder_input, decoder_input], decoder_target, epochs=10, batch_size=64)


# Train the model
# model.fit([encoder_input, decoder_input], decoder_input, epochs=10, batch_size=64)

Epoch 1/10
(64, 20)
(64, 20, 300) emb
hhe
(64, 20, 300) emb
hhe
(64, 20, 300) decode
h
(64, 20, 300) decode
h
(64, 20)
(64, 20, 300) emb
hhe
(64, 20, 300) decode
h
(64, 20)
(64, 20, 300) emb
hhe
(64, 20, 300) decode
h


I0000 00:00:1741952067.677779    2647 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.0000e+00 - loss: 9.2104
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step - accuracy: 0.0688 - loss: 9.2042
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.3188 - loss: 9.1969
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 0.4047 - loss: 9.1875
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.2773 - loss: 9.1724
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 0.0570 - loss: 9.1365
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 0.0109 - loss: 9.0109
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 0.0047 - loss: 8.5653
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step - accuracy: 0.0023 - loss: 7.7814

KeyboardInterrupt: 

In [ ]:
# Instantiate the model
model = Seq2Seq(VOCAB_SIZE, EMBEDDING_DIM, LSTM_UNITS)

# Compile
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# Generate dummy data (replace with actual dataset)
encoder_input = np.random.randint(0, VOCAB_SIZE, (64, MAX_LEN, 300)).astype(
    "float32"
)  # (batch_size, seq_len)
decoder_input = np.random.randint(0, VOCAB_SIZE, (64, MAX_LEN, 300)).astype(
    "float32"
)  # (batch_size, seq_len)
decoder_target = np.random.randint(0, VOCAB_SIZE, (64, MAX_LEN)).astype(
    "float32"
)  # Shifted targets

# Train the model
model.fit([encoder_input, decoder_input], decoder_target, epochs=10, batch_size=64)


# Train the model
# model.fit([encoder_input, decoder_input], decoder_input, epochs=10, batch_size=64)

Epoch 1/10
(64, 20, 300)
(64, 20, 300, 300) emb
hhe
(64, 20, 300, 300) emb
hhe
(64, 20, 300, 300) decode
h
(64, 20, 300, 300) decode
h
(64, 20, 300)
(64, 20, 300, 300) emb
hhe
(64, 20, 300, 300) decode
h


/home/admin/Desktop/deep_learning/.venv/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['seq2_seq_10/encoder_10/embedding_20/embeddings', 'seq2_seq_10/decoder_10/embedding_21/embeddings'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


(64, 20, 300)
(64, 20, 300, 300) emb
hhe
(64, 20, 300, 300) decode
h
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.0000e+00 - loss: 9.2224
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - accuracy: 0.0016 - loss: 9.0058
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - accuracy: 7.8125e-04 - loss: 8.7940
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - accuracy: 0.0039 - loss: 8.5303
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - accuracy: 0.0031 - loss: 8.2398
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - accuracy: 0.0047 - loss: 7.9563
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step - accuracy: 0.0039 - loss: 7.6855
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step - accuracy: 0.0023 - loss: 7.4511
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step - accuracy: 0.0039 - loss: 7.2762
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - accuracy: 0.0039 - loss: 7.1284


In [ ]:
embedding = Embedding(2000, 150)
embedding(np.random.randint(0, 2000, (4, 200)))

<tf.Tensor: shape=(4, 200, 150), dtype=float32, numpy=
array([[[ 2.04419754e-02,  1.60009526e-02, -1.31375678e-02, ...,
         -1.07511878e-05, -1.35048851e-02,  4.54467870e-02],
        [-1.01278797e-02,  5.40109724e-03,  3.46671417e-03, ...,
         -8.57667997e-03,  5.93228266e-03, -4.12744656e-02],
        [ 3.08898352e-02, -3.07225119e-02, -4.92722169e-02, ...,
          4.44459431e-02, -3.94137725e-02,  6.92601129e-03],
        ...,
        [ 9.28055122e-03,  3.03516500e-02, -2.01506373e-02, ...,
          7.58842379e-03,  4.56899069e-02, -2.25624088e-02],
        [-1.29135624e-02, -4.78681326e-02, -2.04903483e-02, ...,
          1.06521621e-02, -6.81437179e-03,  2.80079283e-02],
        [-4.84454297e-02, -2.90435441e-02,  2.88064368e-02, ...,
          3.91721986e-02,  1.28799565e-02,  4.83855940e-02]],

       [[ 2.02955939e-02,  1.20376348e-02,  1.05824582e-02, ...,
         -4.22127731e-02,  1.32352598e-02,  1.95932724e-02],
        [-4.01685461e-02, -2.91835517e-04,  4.13